In [27]:
from client import *
from openssl import *
from hashlib import sha256
import random
import base64
PIERRE = 0x504945525245
pierre = 88275625857605
FEUILLE =0x464555494C4C45
feuille =19779480974019653
CISEAUX  =0x43495345415558
ciseaux = 18939445432636760

deci = {"PIERRE":0x504945525245,
        "FEUILLE":0x464555494C4C45,
        "CISEAUX":0x43495345415558,
}

def generate_elGamal_key(key):
    p = int('FFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD1'
            '29024E088A67CC74020BBEA63B139B22514A08798E3404DD'
            'EF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245'
            'E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7ED'
            'EE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3D'
            'C2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F'
            '83655D23DCA3AD961C62F356208552BB9ED529077096966D'
            '670C354E4ABC9804F1746C08CA237327FFFFFFFFFFFFFFFF', base=16)
    g = 2
    h = pow(g,int(random.uniform(300,50000)))%p
    try:
        k = post_stp(key,"/bin/banks/assistant/start",{'p':p,'g':g,'h':h})
        print(k)
    except Exception as e:
        print(decrypt_b64(e.msg,key))

    return(p,g,h,k)


def round_pfc(key):
    dict = get_stp(key,"bin/banks/assistant/round")
    return dict

def send_gage(coup,key,g,p,h):
    coup = deci[coup]
    y = int(random.uniform(0,500))%p
    a = pow(g,y) % p
    b = (coup * pow(h,y)) % p
    try:
        dict = post_stp(key,"/bin/banks/assistant/move",{'a':a,'b':b})
    except Exception as e:
        print(decrypt_b64(e.msg,key))
        raise
    return dict,y


def outcome(coup,key,y):
    move = coup.encode('ascii')
    try:
        adv_coup = post_stp(key,"/bin/banks/assistant/outcome",{'move':move,'y':y})
    except Exception as e:
        print(decrypt_b64(e.msg,key))
        raise e
    return adv_coup

def play(key,private_key):
    p,g,h,clef = generate_elGamal_key(key)
    clef = clef.split(',')
    temp = []
    for i in clef:
        temp.append(int(i.split(':')[1].strip(' ').strip('}')))
    print(temp)
    try:
        dict = round_pfc(key)
    except Exception as e:
        print(decrypt_b64(e.msg,key))

    coup = input("Entrez le coup : ")
    adv_gage,y = send_gage(coup,key,g,p,h)
    dict_outcome = outcome()

def get_stp(K,adess):
        dict = {'method':'GET','url':adess}
        jdict =json.dumps(dict)
        enc_jdict = encrypt(jdict,K)
        response = c.post_raw("/bin/gateway",base64.b64decode(enc_jdict))
        return(decrypt_b64(response,K))



def post_stp(K,adress,args):
        dict = {'method':'POST','url':adress,'args':args}
        jdict =json.dumps(dict)
        enc_jdict = encrypt(jdict,K)
        response = c.post_raw("/bin/gateway",base64.b64decode(enc_jdict))
        return(decrypt_b64(response,K))



def dh_auten(username):
    k,s = genpkey()
    print(c.post("/bin/key-management/upload-pk",public_key = k,confirm=True))
    parameters = c.get("/bin/login/dh/parameters")
    g = parameters['g']
    p = parameters['p']
    certificate = c.get("/bin/banks/CA")
    x = int(random.uniform(0,500))
    A = pow(g,x)%p
    dic = c.post("/bin/login/dh",username=username,A = A)
    pk = get_pub_cert(certificate)
    B = dic['B']
    k = dic['k']
    S = "{},{},{},{}".format(A,B,k,username)
    verif = verify_signature(pk,dic['signature'],S)
    if(verif == False):
        print("verification invalide")
        return
    AB = pow(B,x)%p
    H = sha256()
    size = 1 + AB.bit_length() // 8
    H.update(AB.to_bytes(size, byteorder='big'))
    K = H.hexdigest()
    print(K)
    adresse = "/bin/login/dh/confirmation"
    T = "{},{},{},{}".format(A,B,k,"UGLIX")
    signature = sign(T,s)
    signature = base64.b64encode(signature).decode()
    post_stp(K,adresse,{'signature':signature})
    return K,k,s


In [38]:
c = Connection()
print(c.post("/bin/login",user="wilkinsonethan",password="!r3YPa7u#&"))
log  = c.get("/bin/login/CHAP")
plaintext = "wilkinsonethan" + "-" + log['challenge']
response = encrypt(plaintext,"!r3YPa7u#&")
print(c.post("/bin/login/CHAP",user = "wilkinsonethan",response = response))
c.post("/bin/jukebox/disable")
c.post("/bin/jukebox/stop")

PAP login successful. Access to /home/wilkinsonethan granted.
CHAP login successful. Access to /home/wilkinsonethan granted.


'Music stopped.'

In [39]:
password = "means"
username = "shane90"
log  = c.get("/bin/login/CHAP")
plaintext = username + "-" + log['challenge']
response = encrypt(plaintext,password)
print(c.post("/bin/login/CHAP",user = username,response = response))

CHAP login successful. Access to /home/shane90 granted.


In [40]:
K,k,s = dh_auten(username)
print(get_stp(K,"/bin/banks/drh"))
print(get_stp(K,"/bin/banks/drh/rules"))

OK
2c6686ffb5b93a618c8bf5feed3632e1769194f90467ec59ff559d968970a93d
La UGLIX Bank est toujours à la recherche de nouveaux talents. Votre candidature
nous semble particulièrement prometteuse en tout points. Cependant, il faut que 
vous sachiez que seuls les meilleurs nous intéressent. C'est pourquoi nous 
pratiquons un petit... test d'embauche. Oh, c'est presque une formalité. Il 
s'agit de jouer à "pierre-feuille-ciseau" en ligne avec mon assistant.

D'ailleurs, je vais vous le présenter tout de suite :

ASSISTANT v1.0 : GRRRRRRRRRRRR KILL KILL KILL KILL KILL KILL KILL BWAAAAARRFFFFF

Comme vous pouvez le constater, il est en pleine forme. J'aurais préféré qu'il 
vous explique le jeu lui-même, mais j'ai un peu peur qu'il ne s'en sorte pas.
Aussi, je vous redirige vers le fichier :

	/bin/banks/drh/rules

Vous devez le battre, puis revenir me voir. Au fait, pour vous aider, vous avez
le droit à ça :

    BANK.ACCESS:2331:3@375715|67d5baaecd849c0266c971a883aa8090
Pour pouvoir jouer en li

In [41]:
p,g,h,clef = generate_elGamal_key(K)

{"h": 1338854666338551042222525539369668732583492936931643787700793254185665683185203170505781606223675228741146975820724868165893903008188787835603823182733618355554781904698113390558880492889316471132705246912261382509702251425993828777945852349884306938834451341952741957614157222207230334049319890657932643373961997814116920007928840205805935768893109662767194386203482991471308472152210664977349175186987700027293751287039897584962763143138506746423853646997764969, "p": 2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919, "g": 2}


In [42]:
clef2 = clef.split(',')
temp = []
for i in clef2:
    temp.append(int(i.split(':')[1].strip(' ').strip('}')))
try:
    dict = round_pfc(K)
except Exception as e:
    print(decrypt_b64(e.msg,K))

[1338854666338551042222525539369668732583492936931643787700793254185665683185203170505781606223675228741146975820724868165893903008188787835603823182733618355554781904698113390558880492889316471132705246912261382509702251425993828777945852349884306938834451341952741957614157222207230334049319890657932643373961997814116920007928840205805935768893109662767194386203482991471308472152210664977349175186987700027293751287039897584962763143138506746423853646997764969, 2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919, 2]
uFS filesystem error: bin/banks/assistant/round not found.


In [43]:
print(get_stp(K,"bin/banks/assistant/round"))

ServerError: ERREUR 404, b"Salted__\xcak\xf5\xde\xdc\xbeF\xbb\x14\xa2\xaf\xe9\x14\x16\x88c\x87\xb5\xc6GF\x98\rG\xd0\xc6+W\x82\xc5&\xea[!\xb9\xa8\x16m\x94 \xb2\xa3\xe4z\xda\xbaj\xea\xc7CxH\xcd9\x9cp\xfbUF8=,\xac\xa3\xee\xd1\x18|\x1e\xca'\xb9"